In [219]:
import pickle 
import pandas as pd 
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder


In [220]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [221]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [222]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [223]:
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [224]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [225]:
## combine the both original data 

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)


In [226]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [227]:
x  = data.drop('Exited',axis=1)
y = data['Exited']

In [228]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [229]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


### ANN Regression Problem Statement 

In [230]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 


In [231]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)   ## Output layers for regression    


])

## compile the model 

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

C:\Users\Dinesh\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [232]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime 

## Setup TensorBoard 

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboards_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [233]:
## set up Early stopping 

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [234]:
## Train the models 

history  =  model.fit(
    x_train,y_train,
    validation_data= (x_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboards_callback]
)



Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2493 - mae: 0.2493 - val_loss: 0.2121 - val_mae: 0.2121
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2115 - mae: 0.2115 - val_loss: 0.1974 - val_mae: 0.1974
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1954 - mae: 0.1954 - val_loss: 0.1848 - val_mae: 0.1848
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1867 - mae: 0.1867 - val_loss: 0.1818 - val_mae: 0.1818
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1800 - mae: 0.1800 - val_loss: 0.1747 - val_mae: 0.1747
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1753 - mae: 0.1753 - val_loss: 0.1715 - val_mae: 0.1715
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1713 - mae: 0.1713 - val_loss: 0.1682 - val_mae: 0.1682
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1683 - mae: 0.1683 - val_loss: 0.1680 - val_mae: 0.1680
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/

In [238]:
!pkill -f tensorboard


'pkill' is not recognized as an internal or external command,
operable program or batch file.


In [241]:
%reload_ext tensorboard
%tensorboard --logdir regressionlogs/fit --reload_interval 5


ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [242]:
## Evaluate model on the test data 

test_loss,test_mae = model.evaluate(x_test,y_test)

print(f'Test Mae : {test_mae}')
print(f'Test loss : {test_loss}')


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1444 - mae: 0.1444
Test Mae : 0.14441882073879242
Test loss : 0.14441882073879242


In [237]:
## save the regression model 

model.save('regression.h5')